# Selection of Stocks into a Portfilio #

In the current document we show how you can pick the best stock and strategy combinations out of a big collecion from a universe.

## Setup ##
### Import Libraries ###

In [ ]:
%classpath config resolver maven-public http://software.pschatzmann.ch/repository/maven-public/
%classpath add mvn ch.pschatzmann:investor:0.9-SNAPSHOT
%classpath add mvn ch.pschatzmann:jupyter-jdk-extensions:0.0.1-SNAPSHOT


Added new repo: maven-public


### Imports ###

In [ ]:
// our stock evaluation framwork
import ch.pschatzmann.dates._;
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.data.universe._;
import ch.pschatzmann.stocks.input._;
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.execution._;
import ch.pschatzmann.stocks.execution.fees._;
import ch.pschatzmann.stocks.execution.price._;
import ch.pschatzmann.stocks.parameters._;
import ch.pschatzmann.stocks.strategy._;
import ch.pschatzmann.stocks.strategy.optimization._;
import ch.pschatzmann.stocks.strategy.allocation._;
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.integration._;
import ch.pschatzmann.stocks.integration.ChartData.FieldName._;
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._;

// java
import java.util.stream.Collectors;
import java.util._;
//import java.util.function.Consumer;

/// jupyter custom displayer
import ch.pschatzmann.display.Displayers


### Setup Logging ###

In [ ]:
Displayers.setup("WARN")


## Selection of all Stocks and Strategies ##


First we need to specify the strategies that we want to use for the evaluation. We just use all:

In [ ]:
TradingStrategyFactory.list()

For defining the selection of stocks we use a universe 


In [ ]:
new MarketUniverse("NASDAQ")


and we define the optimization period as 2016-01-01-2016-21-31 and the trading period as
2017-01-01 to today:


In [ ]:
Context.getDateRanges("2016-01-01","2017-01-01");


### Run the Selection  - 1st Attempt ###

We define a StrategySelector and feed it to the StockSelector.
The optional Restartable parameter is making sure that we save the temporary result every 50 records so that we do not need to reprocess them when we need to restart the functionality.

We store the result in a file and make sure that we do not run the selection again if the file already exists.


In [ ]:
// setup of the model
var periods = Context.getDateRanges("2016-01-01","2017-01-01");
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
var universe = new MarketUniverse("NASDAQ")
var strategies = TradingStrategyFactory.list()
var strategySelector = new StrategySelector(account, strategies, periods.get(0), KPI.AbsoluteReturn)
var stockSelector = new StockSelector(strategySelector, new Restartable("./temp/restart-v1.ser",50))
// calculate the result
var result = stockSelector.getSelection(200, universe, new MarketArchiveHttpReader())

"**END**"

### Evaluate the Selection ###

Here is the result of the selection:

In [ ]:
Displayers.display(Context.tail(result.getResult(),10));


Finally we use the selected result as our portfolio and we can run a simulation on it for the
last year:

In [ ]:
var account = new Account("Simulation","USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(10.0));
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader);
var executor = new StrategyExecutor(trader, allocationStrategy);
executor.addStrategy(result.getStrategies(new MarketArchiveHttpReader()));
executor.run(periods.get(1));

println(account.getKPIValues())


In [ ]:
Displayers.display(account.getTransactions().collect(Collectors.toList()));


In [ ]:
var chart = new TimeSeriesChart()
chart.add(account.getTotalValueHistory(),"")
Displayers.display(chart.displayChart())


In [ ]:
import scala.collection.JavaConversions._

var chart1 = new TimeSeriesChart()
chart1.add(account.getCashHistoryForAllDates(), "Cash");
chart1.add(account.getTotalValueHistory(), "Total Value");
chart1.add(account.getActualValueHistory(), "ActualValue");
for (id <- account.getStockIDs()) {
    var data = account.getActualValueHistory(id);
    chart1.add(data,id.toString())
}
//chart1.addLabels(account.getTransactions())
Displayers.display(chart1.displayCharts())

Though at first sight it seems to be promising, the result is in fact quite disapointing: We only get 2 stocks which are used for trading and they are just bought and never sold. So it is just by luck that we were successfull!

### Run the Selection  - 2nd Attempt ###


On our 2nd trial we want to use the followig additional criteria:
- no penny stocks: rate > 5.0
- activly traded: number of trades >= 4

Therefore we define the following predicates which drive our selection

In [ ]:
import ch.pschatzmann.stocks.strategy.selection.SelectionState;
import java.util.function.Predicate;

class NumberOfTradesPredicate( numberOfTrades: Double) extends Predicate[SelectionState]  {
     override def test(state:SelectionState):scala.Boolean = {
        return state.result().getDouble(KPI.NumberOfTrades) > numberOfTrades;
    };
}

class PriceLimitPredicate(account:Account, date:Date, limit: Double) extends Predicate[SelectionState]  {
     override def test(state:SelectionState):scala.Boolean = {
        return account.getStockPrice(state.getStockID(), date)>limit;
    };
}

In [ ]:

// setup of the model

var periods = Context.getDateRanges("2016-01-01","2017-01-01");
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
var universe = new MarketUniverse("NASDAQ")
var strategies = TradingStrategyFactory.list()
var predicate = new NumberOfTradesPredicate(4.0).and(new PriceLimitPredicate(account, periods.get(0).getStart(), 5.0));
var strategySelector = new StrategySelector(account, strategies, periods.get(0), KPI.AbsoluteReturn, predicate)
var stockSelector = new StockSelector(strategySelector, new Restartable("./temp/restart-v1A.ser",50))
// calculate the result
var result = stockSelector.getSelection(200, universe, new MarketArchiveHttpReader())
// save the result to a file
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader);
var executor = new StrategyExecutor(trader, allocationStrategy);
executor.addStrategy(result.getStrategies(new MarketArchiveHttpReader()));
executor.run(periods.get(1));

account.getKPIValues()

### Selection with Sharpe Ratio ###

In the long run we might be better off by using the Sharpe Ratio instead of the Absolute Return as selection criterium. So here is our next try:


In [ ]:
// setup of the model

var periods = Context.getDateRanges("2016-01-01","2017-01-01");
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
var universe = new MarketUniverse("NASDAQ")
var strategies = TradingStrategyFactory.list()
var strategySelector = new StrategySelector(account, strategies, periods.get(0), KPI.SharpeRatio)
var stockSelector = new StockSelector(strategySelector, new Restartable("./temp/restart-v1S.ser",50))
// calculate the result
var result = stockSelector.getSelection(200, universe, new MarketArchiveHttpReader())
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader);
var executor = new StrategyExecutor(trader, allocationStrategy);
executor.addStrategy(result.getStrategies(new MarketArchiveHttpReader()));
executor.run(periods.get(1));

account.getKPIValues()

## Electronic Monkey ##

We can compare the results above with an Monkey Picker which just pick 50 random stocks and strategies 

In [ ]:
var account = new Account("Simulation","USD", 1000000.00, periods.get(0).getStart(), new PerTradeFees(10.0));
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader);
var executor = new StrategyExecutor(trader, allocationStrategy);
var randomStrategies = TradingStrategyFactory.getRandomStrategies(new MarketUniverse("NASDAQ"),new MarketArchiveHttpReader(), 50)
executor.addStrategy(randomStrategies);
executor.run(periods.get(1));

println(account.getKPIValues())

## Optimization ##
We can also evaluate if parameter optimization makes a difference. We optimize the parameters based
on the closing rates of the year 2015. Then we select the stocks that perfomred best in 2016.
Finally we evaluate the performance for 2017 

In [ ]:
// setup of the model in background
var periods = Context.getDateRanges("2015-01-01","2016-01-01","2017-01-01","2018-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
// run selection
var universe = new MarketUniverse("NASDAQ")
var strategies = TradingStrategyFactory.list()
var optimizer = new BinarySearchOptimizer(new SimulatedFitness(account), KPI.AbsoluteReturn)
var strategySelector = new StrategySelectorOptimized(account, strategies, periods.get(0), periods.get(1), optimizer)
var stockSelector = new StockSelector(strategySelector, new Restartable("./temp/restart-v2.ser",20))
// calculate the result
var resultOptimized = stockSelector.getSelection(200, universe, new MarketArchiveHttpReader())
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader);
var executor = new StrategyExecutor(trader, allocationStrategy);
executor.addStrategy(resultOptimized.getStrategies(new MarketArchiveHttpReader()));
executor.run(periods.get(2));

account.getKPIValues()

In [ ]:
Displayers.display(account.getTransactions());
